In [52]:
import findspark
findspark.init()
import pyspark
findspark.find()
from pyspark.sql.types import MapType,StringType,IntegerType,DateType
from pyspark.sql.functions import col, to_timestamp

In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
conf = pyspark.SparkConf().setAppName('unir').setMaster('local')
sc = pyspark.SparkContext(conf=conf)

In [3]:
from pyspark.sql import HiveContext
sqlContext = HiveContext(sc)
sqlContext.sql("use default")
sqlContext.sql("show tables").show()

C:\spark\python\pyspark\sql\context.py:601: FutureWarning: HiveContext is deprecated in Spark 2.0.0. Please use SparkSession.builder.enableHiveSupport().getOrCreate() instead.
  warnings.warn(


+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|  default|    sales|      false|
+---------+---------+-----------+



In [68]:
sqlContext.sql("CREATE SCHEMA IF NOT EXISTS Stage;")
sqlContext.sql("DROP TABLE IF EXISTS Stage.Sales")
sqlSales = """
CREATE TABLE IF NOT EXISTS Stage.Sales
(Store INT,Dept INT,Date String,Weekly_Sales DECIMAL(10,2),IsHoliday BOOLEAN)
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
TBLPROPERTIES ('skip.header.line.count'='1')
"""
sqlContext.sql(sqlSales)
sqlContext.sql("DROP TABLE IF EXISTS Sales")
sqlSales = """
CREATE TABLE IF NOT EXISTS Sales
(Store INT,Dept INT,Date TIMESTAMP,Weekly_Sales DECIMAL(10,2),IsHoliday BOOLEAN)
STORED AS ORC
"""
sqlContext.sql(sqlSales)
sqlContext.sql("use stage")
sqlContext.sql("show tables").show()
sqlContext.sql("use default")
sqlContext.sql("show tables").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|    stage|    sales|      false|
|    stage|    store|      false|
+---------+---------+-----------+

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|  default|    sales|      false|
|  default|    store|      false|
+---------+---------+-----------+



In [76]:
sqlContext.sql("DROP TABLE IF EXISTS Stage.Store")
sqlSales = """
CREATE TABLE IF NOT EXISTS Stage.Store
(Store Int,Type String, Size Int)
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
TBLPROPERTIES ('skip.header.line.count'='1')
"""
sqlContext.sql(sqlSales)
sqlContext.sql("DROP TABLE IF EXISTS Store")
sqlSales = """
CREATE EXTERNAL TABLE IF NOT EXISTS Store
(Store Int,Type String, Size Int)
STORED AS ORC
location '/analytics_equipo_24/external_store_data'
"""
sqlContext.sql(sqlSales)
sqlContext.sql("use stage")
sqlContext.sql("show tables").show()
sqlContext.sql("use default")
sqlContext.sql("show tables").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|    stage|    sales|      false|
|    stage|    store|      false|
+---------+---------+-----------+

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|  default|    sales|      false|
|  default|    store|      false|
+---------+---------+-----------+



In [78]:
sqlContext.sql("DROP TABLE IF EXISTS Stage.Features")
sqlSales = """
CREATE TABLE IF NOT EXISTS Stage.Features
(Store int, Date string, Temperature decimal(10,2), Fuel_Price decimal(10,2),
MarkDown1 decimal(10,2), MarkDown2 decimal(10,2), MarkDown3 decimal(10,2), MarkDown4 decimal(10,2), MarkDown5 decimal(10,2),
CPI decimal(10,2), Unemployment decimal(10,2), IsHoliday boolean)
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
TBLPROPERTIES ('skip.header.line.count'='1')
"""
sqlContext.sql(sqlSales)
sqlContext.sql("DROP TABLE IF EXISTS Features")
sqlSales = """
CREATE TABLE IF NOT EXISTS Features
(Store int, Date string, Temperature decimal(10,2), Fuel_Price decimal(10,2),
MarkDown1 decimal(10,2), MarkDown2 decimal(10,2), MarkDown3 decimal(10,2), MarkDown4 decimal(10,2), MarkDown5 decimal(10,2),
CPI decimal(10,2), Unemployment decimal(10,2), IsHoliday boolean)
STORED AS ORC
"""
sqlContext.sql(sqlSales)
sqlContext.sql("use stage")
sqlContext.sql("show tables").show()
sqlContext.sql("use default")
sqlContext.sql("show tables").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|    stage| features|      false|
|    stage|    sales|      false|
|    stage|    store|      false|
+---------+---------+-----------+

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|  default| features|      false|
|  default|    sales|      false|
|  default|    store|      false|
+---------+---------+-----------+



In [8]:
import subprocess

path = "/analytics_equipo_24"
hdfs_path = "c:\\winutils\\bin\\"
result = subprocess.run(f"{hdfs_path}hdfs dfs -mkdir {path}", shell=True, capture_output=True)
result.stdout

b''

In [70]:
result = subprocess.run(f"{hdfs_path}hdfs dfs -copyFromLocal c:\\repo\\unir\\*.csv {path}", shell=True, capture_output=True)
result = subprocess.run(f"{hdfs_path}hdfs dfs -ls {path}", shell=True, capture_output=True)
result.stdout.splitlines()

[b'Found 3 items',
 b'-rw-r--r--   1 mcast supergroup     600478 2022-02-19 19:26 /analytics_equipo_24/features.csv',
 b'-rw-r--r--   1 mcast supergroup   13264115 2022-02-19 20:33 /analytics_equipo_24/sales.csv',
 b'-rw-r--r--   1 mcast supergroup        577 2022-02-19 19:26 /analytics_equipo_24/stores.csv']

In [71]:
#Insertar data de sales.csv a tabla de staging
sqlContext.sql(f"LOAD DATA INPATH 'hdfs://localhost:9000/{path}/sales.csv' INTO TABLE stage.sales;")
stage_sales = sqlContext.table("stage.sales")
stage_sales.show()

+-----+----+----------+------------+---------+
|Store|Dept|      Date|Weekly_Sales|IsHoliday|
+-----+----+----------+------------+---------+
| null|null|      Date|        null|     null|
|    1|   1|05/02/2010|    24924.50|    false|
|    1|   1|12/02/2010|    46039.49|     true|
|    1|   1|19/02/2010|    41595.55|    false|
|    1|   1|26/02/2010|    19403.54|    false|
|    1|   1|05/03/2010|    21827.90|    false|
|    1|   1|12/03/2010|    21043.39|    false|
|    1|   1|19/03/2010|    22136.64|    false|
|    1|   1|26/03/2010|    26229.21|    false|
|    1|   1|02/04/2010|    57258.43|    false|
|    1|   1|09/04/2010|    42960.91|    false|
|    1|   1|16/04/2010|    17596.96|    false|
|    1|   1|23/04/2010|    16145.35|    false|
|    1|   1|30/04/2010|    16555.11|    false|
|    1|   1|07/05/2010|    17413.94|    false|
|    1|   1|14/05/2010|    18926.74|    false|
|    1|   1|21/05/2010|    14773.04|    false|
|    1|   1|28/05/2010|    15580.43|    false|
|    1|   1|0

In [83]:
#Inserta data de store.csv a tabla de staging
sqlContext.sql(f"LOAD DATA INPATH 'hdfs://localhost:9000/{path}/stores.csv' INTO TABLE stage.store;")
stage_store = sqlContext.table("stage.store")
stage_store.show()

+-----+----+------+
|Store|Type|  Size|
+-----+----+------+
| null|Type|  null|
|    1|   A|151315|
|    2|   A|202307|
|    3|   B| 37392|
|    4|   A|205863|
|    5|   B| 34875|
|    6|   A|202505|
|    7|   B| 70713|
|    8|   A|155078|
|    9|   B|125833|
|   10|   B|126512|
|   11|   A|207499|
|   12|   B|112238|
|   13|   A|219622|
|   14|   A|200898|
|   15|   B|123737|
|   16|   B| 57197|
|   17|   B| 93188|
|   18|   B|120653|
|   19|   A|203819|
+-----+----+------+
only showing top 20 rows



In [84]:
#Inserta data de features.csv a tabla de stagin
sqlContext.sql(f"LOAD DATA INPATH 'hdfs://localhost:9000/{path}/features.csv' INTO TABLE stage.features;")
stage_features = sqlContext.table("stage.features")
stage_features.show()

+-----+----------+-----------+----------+---------+---------+---------+---------+---------+------+------------+---------+
|Store|      Date|Temperature|Fuel_Price|MarkDown1|MarkDown2|MarkDown3|MarkDown4|MarkDown5|   CPI|Unemployment|IsHoliday|
+-----+----------+-----------+----------+---------+---------+---------+---------+---------+------+------------+---------+
| null|      Date|       null|      null|     null|     null|     null|     null|     null|  null|        null|     null|
|    1|05/02/2010|      42.31|      2.57|     null|     null|     null|     null|     null|211.10|        8.11|    false|
|    1|12/02/2010|      38.51|      2.55|     null|     null|     null|     null|     null|211.24|        8.11|     true|
|    1|19/02/2010|      39.93|      2.51|     null|     null|     null|     null|     null|211.29|        8.11|    false|
|    1|26/02/2010|      46.63|      2.56|     null|     null|     null|     null|     null|211.32|        8.11|    false|
|    1|05/03/2010|      

In [72]:
stage_sales = sqlContext.table("stage.sales")
#Quitar headers, bug conocido de Hive con PySpark
stage_sales = sqlContext.createDataFrame(stage_sales.tail(stage_sales.count()-1), stage_sales.schema)
#Convertir campo Date de String type a Date type
stage_sales = stage_sales.withColumn("Date", to_timestamp('Date', 'dd/MM/yyyy'))
stage_sales.show()
stage_sales.printSchema()

+-----+----+-------------------+------------+---------+
|Store|Dept|               Date|Weekly_Sales|IsHoliday|
+-----+----+-------------------+------------+---------+
|    1|   1|2010-02-05 00:00:00|    24924.50|    false|
|    1|   1|2010-02-12 00:00:00|    46039.49|     true|
|    1|   1|2010-02-19 00:00:00|    41595.55|    false|
|    1|   1|2010-02-26 00:00:00|    19403.54|    false|
|    1|   1|2010-03-05 00:00:00|    21827.90|    false|
|    1|   1|2010-03-12 00:00:00|    21043.39|    false|
|    1|   1|2010-03-19 00:00:00|    22136.64|    false|
|    1|   1|2010-03-26 00:00:00|    26229.21|    false|
|    1|   1|2010-04-02 00:00:00|    57258.43|    false|
|    1|   1|2010-04-09 00:00:00|    42960.91|    false|
|    1|   1|2010-04-16 00:00:00|    17596.96|    false|
|    1|   1|2010-04-23 00:00:00|    16145.35|    false|
|    1|   1|2010-04-30 00:00:00|    16555.11|    false|
|    1|   1|2010-05-07 00:00:00|    17413.94|    false|
|    1|   1|2010-05-14 00:00:00|    18926.74|   

In [91]:
#Insertar datos limpios a tabla final para "sales"
stage_sales.write.format("hive").mode("overwrite").saveAsTable("default.sales")
sales = sqlContext.table("sales")
sales.show()

+-----+----+-------------------+------------+---------+
|Store|Dept|               Date|Weekly_Sales|IsHoliday|
+-----+----+-------------------+------------+---------+
|    1|   1|2010-02-05 00:00:00|    24924.50|    false|
|    1|   1|2010-02-12 00:00:00|    46039.49|     true|
|    1|   1|2010-02-19 00:00:00|    41595.55|    false|
|    1|   1|2010-02-26 00:00:00|    19403.54|    false|
|    1|   1|2010-03-05 00:00:00|    21827.90|    false|
|    1|   1|2010-03-12 00:00:00|    21043.39|    false|
|    1|   1|2010-03-19 00:00:00|    22136.64|    false|
|    1|   1|2010-03-26 00:00:00|    26229.21|    false|
|    1|   1|2010-04-02 00:00:00|    57258.43|    false|
|    1|   1|2010-04-09 00:00:00|    42960.91|    false|
|    1|   1|2010-04-16 00:00:00|    17596.96|    false|
|    1|   1|2010-04-23 00:00:00|    16145.35|    false|
|    1|   1|2010-04-30 00:00:00|    16555.11|    false|
|    1|   1|2010-05-07 00:00:00|    17413.94|    false|
|    1|   1|2010-05-14 00:00:00|    18926.74|   

In [86]:
stage_store = sqlContext.table("stage.store")
#Quitar headers, bug conocido de Hive con PySpark
stage_store = sqlContext.createDataFrame(stage_store.tail(stage_store.count()-1), stage_store.schema)
stage_store.show()
stage_store.printSchema()

+-----+----+------+
|Store|Type|  Size|
+-----+----+------+
|    1|   A|151315|
|    2|   A|202307|
|    3|   B| 37392|
|    4|   A|205863|
|    5|   B| 34875|
|    6|   A|202505|
|    7|   B| 70713|
|    8|   A|155078|
|    9|   B|125833|
|   10|   B|126512|
|   11|   A|207499|
|   12|   B|112238|
|   13|   A|219622|
|   14|   A|200898|
|   15|   B|123737|
|   16|   B| 57197|
|   17|   B| 93188|
|   18|   B|120653|
|   19|   A|203819|
|   20|   A|203742|
+-----+----+------+
only showing top 20 rows

root
 |-- Store: integer (nullable = true)
 |-- Type: string (nullable = true)
 |-- Size: integer (nullable = true)



In [88]:
#Insertar datos limpios a tabla final para "store"
stage_store.write.format("hive").mode("overwrite").saveAsTable("default.store")
store = sqlContext.table("store")
store.show()

+-----+----+------+
|Store|Type|  Size|
+-----+----+------+
|    1|   A|151315|
|    2|   A|202307|
|    3|   B| 37392|
|    4|   A|205863|
|    5|   B| 34875|
|    6|   A|202505|
|    7|   B| 70713|
|    8|   A|155078|
|    9|   B|125833|
|   10|   B|126512|
|   11|   A|207499|
|   12|   B|112238|
|   13|   A|219622|
|   14|   A|200898|
|   15|   B|123737|
|   16|   B| 57197|
|   17|   B| 93188|
|   18|   B|120653|
|   19|   A|203819|
|   20|   A|203742|
+-----+----+------+
only showing top 20 rows



In [92]:
stage_features = sqlContext.table("stage.features")
#Quitar headers, bug conocido de Hive con PySpark
stage_features = sqlContext.createDataFrame(stage_features.tail(stage_features.count()-1), stage_features.schema)
#Convertir campo Date de String type a Date type
stage_features = stage_features.withColumn("Date", to_timestamp('Date', 'dd/MM/yyyy'))
stage_features.show()
stage_features.printSchema()

+-----+-------------------+-----------+----------+---------+---------+---------+---------+---------+------+------------+---------+
|Store|               Date|Temperature|Fuel_Price|MarkDown1|MarkDown2|MarkDown3|MarkDown4|MarkDown5|   CPI|Unemployment|IsHoliday|
+-----+-------------------+-----------+----------+---------+---------+---------+---------+---------+------+------------+---------+
|    1|2010-02-05 00:00:00|      42.31|      2.57|     null|     null|     null|     null|     null|211.10|        8.11|    false|
|    1|2010-02-12 00:00:00|      38.51|      2.55|     null|     null|     null|     null|     null|211.24|        8.11|     true|
|    1|2010-02-19 00:00:00|      39.93|      2.51|     null|     null|     null|     null|     null|211.29|        8.11|    false|
|    1|2010-02-26 00:00:00|      46.63|      2.56|     null|     null|     null|     null|     null|211.32|        8.11|    false|
|    1|2010-03-05 00:00:00|      46.50|      2.63|     null|     null|     null|   

In [95]:
#Insertar datos limpios a tabla final para "features"
stage_features.write.format("hive").mode("overwrite").saveAsTable("default.features")
features = sqlContext.table("features")
features.show()

+-----+-------------------+-----------+----------+---------+---------+---------+---------+---------+------+------------+---------+
|Store|               Date|Temperature|Fuel_Price|MarkDown1|MarkDown2|MarkDown3|MarkDown4|MarkDown5|   CPI|Unemployment|IsHoliday|
+-----+-------------------+-----------+----------+---------+---------+---------+---------+---------+------+------------+---------+
|    1|2010-02-05 00:00:00|      42.31|      2.57|     null|     null|     null|     null|     null|211.10|        8.11|    false|
|    1|2010-02-12 00:00:00|      38.51|      2.55|     null|     null|     null|     null|     null|211.24|        8.11|     true|
|    1|2010-02-19 00:00:00|      39.93|      2.51|     null|     null|     null|     null|     null|211.29|        8.11|    false|
|    1|2010-02-26 00:00:00|      46.63|      2.56|     null|     null|     null|     null|     null|211.32|        8.11|    false|
|    1|2010-03-05 00:00:00|      46.50|      2.63|     null|     null|     null|   

In [105]:
sqlContext.sql("SELECT COUNT(1) AS ConteoRegistrosSales FROM sales").show()
sqlContext.sql("SELECT COUNT(1) AS ConteoRegistrosStore FROM store").show()
sqlContext.sql("SELECT COUNT(1) AS ConteoRegistrosFeatures FROM features").show()

+--------------------+
|ConteoRegistrosSales|
+--------------------+
|              421570|
+--------------------+

+--------------------+
|ConteoRegistrosStore|
+--------------------+
|                  45|
+--------------------+

+-----------------------+
|ConteoRegistrosFeatures|
+-----------------------+
|                   8190|
+-----------------------+



In [121]:
#	Estudiar el rango (máximo y mínimo) de cada variable numérica. Indicar estos datos en el informe.
sqlContext.sql("""
SELECT 'Weekly_Sales' AS Metric, 'Max' AS Type, Max(Weekly_Sales) as Value, 'sales' as Table FROM sales
UNION ALL
SELECT 'Weekly_Sales' AS Metric, 'Min' AS Type, Min(Weekly_Sales) as Value, 'sales' as Table FROM sales
""").show()
sqlContext.sql("""
SELECT 'Size' AS Metric, 'Max' AS Type, Max(Size) as Value, 'store' as Table FROM store
UNION ALL
SELECT 'Size' AS Metric, 'Min' AS Type, Min(Size) as Value, 'store' as Table FROM store
""").show()
sqlContext.sql("""
SELECT 'Temperature' AS Metric, 'Max' AS Type, Max(Temperature) as Value, 'feature' as Table FROM features
UNION ALL
SELECT 'Temperature' AS Metric, 'Min' AS Type, Min(Temperature) as Value, 'feature' as Table FROM features
UNION ALL
SELECT 'Fuel_Price' AS Metric, 'Max' AS Type, Max(Fuel_Price) as Value, 'feature' as Table FROM features
UNION ALL
SELECT 'Fuel_Price' AS Metric, 'Min' AS Type, Min(Fuel_Price) as Value, 'feature' as Table FROM features
UNION ALL
SELECT 'MarkDown1' AS Metric, 'Max' AS Type, Max(MarkDown1) as Value, 'feature' as Table FROM features
UNION ALL
SELECT 'MarkDown1' AS Metric, 'Min' AS Type, Min(MarkDown1) as Value, 'feature' as Table FROM features
UNION ALL
SELECT 'MarkDown2' AS Metric, 'Max' AS Type, Max(MarkDown2) as Value, 'feature' as Table FROM features
UNION ALL
SELECT 'MarkDown2' AS Metric, 'Min' AS Type, Min(MarkDown2) as Value, 'feature' as Table FROM features
UNION ALL
SELECT 'MarkDown3' AS Metric, 'Max' AS Type, Max(MarkDown3) as Value, 'feature' as Table FROM features
UNION ALL
SELECT 'MarkDown3' AS Metric, 'Min' AS Type, Min(MarkDown3) as Value, 'feature' as Table FROM features
UNION ALL
SELECT 'MarkDown4' AS Metric, 'Max' AS Type, Max(MarkDown4) as Value, 'feature' as Table FROM features
UNION ALL
SELECT 'MarkDown4' AS Metric, 'Min' AS Type, Min(MarkDown4) as Value, 'feature' as Table FROM features
UNION ALL
SELECT 'MarkDown5' AS Metric, 'Max' AS Type, Max(MarkDown5) as Value, 'feature' as Table FROM features
UNION ALL
SELECT 'MarkDown5' AS Metric, 'Min' AS Type, Min(MarkDown5) as Value, 'feature' as Table FROM features
UNION ALL
SELECT 'CPI' AS Metric, 'Max' AS Type, Max(CPI) as Value, 'feature' as Table FROM features
UNION ALL
SELECT 'CPI' AS Metric, 'Min' AS Type, Min(CPI) as Value, 'feature' as Table FROM features
UNION ALL
SELECT 'Unemployment' AS Metric, 'Max' AS Type, Max(Unemployment) as Value, 'feature' as Table FROM features
UNION ALL
SELECT 'Unemployment' AS Metric, 'Min' AS Type, Min(Unemployment) as Value, 'feature' as Table FROM features
""").show()

+------------+----+---------+-----+
|      Metric|Type|    Value|Table|
+------------+----+---------+-----+
|Weekly_Sales| Max|693099.36|sales|
|Weekly_Sales| Min| -4988.94|sales|
+------------+----+---------+-----+

+------+----+------+-----+
|Metric|Type| Value|Table|
+------+----+------+-----+
|  Size| Max|219622|store|
|  Size| Min| 34875|store|
+------+----+------+-----+

+------------+----+---------+-------+
|      Metric|Type|    Value|  Table|
+------------+----+---------+-------+
| Temperature| Max|   101.95|feature|
| Temperature| Min|    -7.29|feature|
|  Fuel_Price| Max|     4.47|feature|
|  Fuel_Price| Min|     2.47|feature|
|   MarkDown1| Max|103184.98|feature|
|   MarkDown1| Min| -2781.45|feature|
|   MarkDown2| Max|104519.54|feature|
|   MarkDown2| Min|  -265.76|feature|
|   MarkDown3| Max|149483.31|feature|
|   MarkDown3| Min|  -179.26|feature|
|   MarkDown4| Max| 67474.85|feature|
|   MarkDown4| Min|     0.22|feature|
|   MarkDown5| Max|771448.10|feature|
|   MarkDown

In [140]:
#Estudiar las diferentes categorías de las principales variables categóricas y el número de filas correspondientes a cada categoría.
#Indicar las conclusiones en el informe.
sqlContext.sql("""
SELECT Store, COUNT(1) AS Count, 'sales' AS Table FROM sales GROUP BY 1 ORDER BY 2 DESC
""").show(50)
sqlContext.sql("""
SELECT Dept, COUNT(1) AS Count, 'sales' AS Table FROM sales GROUP BY 1 ORDER BY 2 DESC
""").show(100)
sqlContext.sql("""
SELECT date_format(Date,'yyyyMM'), COUNT(1) AS Count, 'sales' AS Table FROM sales GROUP BY 1 ORDER BY 2 DESC
""").show(50)
sqlContext.sql("""
SELECT isHoliday, COUNT(1) AS Count, 'sales' AS Table FROM sales GROUP BY 1 ORDER BY 2 DESC
""").show()

+-----+-----+-----+
|Store|Count|Table|
+-----+-----+-----+
|   13|10474|sales|
|   10|10315|sales|
|    4|10272|sales|
|    1|10244|sales|
|    2|10238|sales|
|   24|10228|sales|
|   27|10225|sales|
|   34|10224|sales|
|   20|10214|sales|
|    6|10211|sales|
|   32|10202|sales|
|   19|10148|sales|
|   31|10142|sales|
|   28|10113|sales|
|   41|10088|sales|
|   11|10062|sales|
|   23|10050|sales|
|   14|10040|sales|
|   40|10017|sales|
|   15| 9901|sales|
|    8| 9895|sales|
|   39| 9878|sales|
|   17| 9864|sales|
|   18| 9859|sales|
|   26| 9854|sales|
|   25| 9804|sales|
|    7| 9762|sales|
|   12| 9705|sales|
|   22| 9688|sales|
|   45| 9637|sales|
|   21| 9582|sales|
|   35| 9528|sales|
|   29| 9455|sales|
|   16| 9443|sales|
|    3| 9036|sales|
|    5| 8999|sales|
|    9| 8867|sales|
|   38| 7362|sales|
|   37| 7206|sales|
|   44| 7169|sales|
|   30| 7156|sales|
|   42| 6953|sales|
|   43| 6751|sales|
|   33| 6487|sales|
|   36| 6222|sales|
+-----+-----+-----+

+----+-----+-----+


In [136]:
sqlContext.sql("""
SELECT Type, COUNT(1) AS Count, 'store' AS Table FROM store GROUP BY 1 ORDER BY 2 DESC
""").show(50)

+----+-----+-----+
|Type|Count|Table|
+----+-----+-----+
|   A|   22|store|
|   B|   17|store|
|   C|    6|store|
+----+-----+-----+



In [139]:
sqlContext.sql("""
SELECT Store, COUNT(1) AS Count, 'features' AS Table FROM features GROUP BY 1 ORDER BY 2 DESC
""").show(50)
sqlContext.sql("""
SELECT date_format(Date,'yyyyMM'), COUNT(1) AS Count, 'features' AS Table FROM features GROUP BY 1 ORDER BY 2 DESC
""").show(50)
sqlContext.sql("""
SELECT isHoliday, COUNT(1) AS Count, 'features' AS Table FROM features GROUP BY 1 ORDER BY 2 DESC
""").show()

+-----+-----+--------+
|Store|Count|   Table|
+-----+-----+--------+
|   31|  182|features|
|   34|  182|features|
|   28|  182|features|
|   26|  182|features|
|   27|  182|features|
|   44|  182|features|
|   12|  182|features|
|   22|  182|features|
|    1|  182|features|
|   13|  182|features|
|    6|  182|features|
|   16|  182|features|
|    3|  182|features|
|   20|  182|features|
|   40|  182|features|
|    5|  182|features|
|   19|  182|features|
|   41|  182|features|
|   15|  182|features|
|   43|  182|features|
|   37|  182|features|
|    9|  182|features|
|   17|  182|features|
|   35|  182|features|
|    4|  182|features|
|    8|  182|features|
|   23|  182|features|
|   39|  182|features|
|    7|  182|features|
|   10|  182|features|
|   45|  182|features|
|   38|  182|features|
|   25|  182|features|
|   24|  182|features|
|   29|  182|features|
|   21|  182|features|
|   32|  182|features|
|   11|  182|features|
|   33|  182|features|
|   14|  182|features|
|   42|  18

In [153]:
#Buscar valores inexistentes o anómalos 
sqlContext.sql("""
SELECT 'Weekly Sales below 0' AS Anomaly, COUNT(1) AS Count, 'sales' AS Table FROM sales WHERE Weekly_Sales<0
UNION ALL 
SELECT 'Date is null' AS Anomaly, COUNT(1) AS Count, 'sales' AS Table FROM sales WHERE Date IS NULL
UNION ALL 
SELECT 'Invalid IsHoliday Flag' AS Anomaly, COUNT(1) AS Count, 'sales' AS Table FROM sales WHERE IsHoliday IS NULL
UNION ALL 
SELECT 'Size below 0' AS Anomaly, COUNT(1) AS Count, 'store' AS Table FROM store WHERE Size<0
UNION ALL 
SELECT 'Date is null' AS Anomaly, COUNT(1) AS Count, 'features' AS Table FROM features WHERE Date IS NULL
UNION ALL 
SELECT 'Invalid IsHoliday Flag' AS Anomaly, COUNT(1) AS Count, 'features' AS Table FROM features WHERE IsHoliday IS NULL
""").show(truncate=False)

+----------------------+-----+--------+
|Anomaly               |Count|Table   |
+----------------------+-----+--------+
|Weekly Sales below 0  |1285 |sales   |
|Date is null          |0    |sales   |
|Invalid IsHoliday Flag|0    |sales   |
|Size below 0          |0    |store   |
|Date is null          |0    |features|
|Invalid IsHoliday Flag|0    |features|
+----------------------+-----+--------+



In [171]:
#Realizar alguna operación join entre las tablas creadas que el equipo considere relevante. Guardar el resultado en vistas 
#Vista para poder mostrar los datos de la tienda (type y size) con los detalles de ventas.  Esto permite realizar analisis de
#ventas por tamaño y tipo de tienda
#La segunda vista desnormaliza los datos, ya que los features de las store se pueden usar para analizar los datos de ventas
sqlContext.sql("""
CREATE VIEW IF NOT EXISTS StoreSales AS
SELECT str.Store, str.Type, str.Size, sls.Dept, sls.Date, sls.Weekly_Sales, sls.IsHoliday
FROM sales sls INNER JOIN store str ON sls.Store = str.Store
""")
sqlContext.sql("""
SELECT * FROM StoreSales
""").show()
sqlContext.sql("""
CREATE VIEW IF NOT EXISTS StoreFeaturesSales AS
SELECT str.Store, str.Type, str.Size, sls.Dept, sls.Date, sls.Weekly_Sales, sls.IsHoliday,
fts.Temperature,fts.Fuel_Price,fts.CPI,fts.Unemployment
FROM sales sls 
INNER JOIN store str ON sls.Store = str.Store
INNER JOIN features fts ON fts.Store = sls.Store AND fts.Date = sls.Date
""")
sqlContext.sql("""
SELECT * FROM StoreFeaturesSales
""").show()


+-----+----+------+----+-------------------+------------+---------+
|Store|Type|  Size|Dept|               Date|Weekly_Sales|IsHoliday|
+-----+----+------+----+-------------------+------------+---------+
|    1|   A|151315|   1|2010-02-05 00:00:00|    24924.50|    false|
|    1|   A|151315|   1|2010-02-12 00:00:00|    46039.49|     true|
|    1|   A|151315|   1|2010-02-19 00:00:00|    41595.55|    false|
|    1|   A|151315|   1|2010-02-26 00:00:00|    19403.54|    false|
|    1|   A|151315|   1|2010-03-05 00:00:00|    21827.90|    false|
|    1|   A|151315|   1|2010-03-12 00:00:00|    21043.39|    false|
|    1|   A|151315|   1|2010-03-19 00:00:00|    22136.64|    false|
|    1|   A|151315|   1|2010-03-26 00:00:00|    26229.21|    false|
|    1|   A|151315|   1|2010-04-02 00:00:00|    57258.43|    false|
|    1|   A|151315|   1|2010-04-09 00:00:00|    42960.91|    false|
|    1|   A|151315|   1|2010-04-16 00:00:00|    17596.96|    false|
|    1|   A|151315|   1|2010-04-23 00:00:00|    

In [197]:
#Vista para obtener las ventas semanales por tipo
sqlContext.sql("""
CREATE VIEW IF NOT EXISTS WeeklySalesByType AS
SELECT 
Type, SUM(Weekly_Sales) SumWeeklySales, MIN(Weekly_Sales) MinWeeklySales, MAX(Weekly_Sales) MaxWeeklySales, 
AVG(Weekly_Sales) AvgWeeklySales, STDDEV(Weekly_Sales) StdWeeklySales
FROM StoreSales 
GROUP BY 1 ORDER BY 1
""")
#Vista para obtener las metricas de las ventas semanales por tamaño
sqlContext.sql("""
CREATE VIEW IF NOT EXISTS WeeklySalesBySize AS
SELECT CASE WHEN (Size)>100000 THEN 'Big Store' ELSE 'Small Store' END  AS Size, AVG(Weekly_Sales) AVGWeeklySales, 
STDDEV(Weekly_Sales) StdWeeklySales
FROM StoreSales GROUP BY 1 ORDER BY 1
""")
#Vista para obtener las metricas de ventas semanales si el dia fue festivo o no
sqlContext.sql("""
CREATE VIEW IF NOT EXISTS WeeklySalesByIsHoliday AS
SELECT IsHoliday, MIN(Weekly_Sales) MinWeeklySales, MAX(Weekly_Sales) MaxWeeklySales, AVG(Temperature) AvgTemperature
FROM StoreFeaturesSales GROUP BY 1 ORDER BY 1
""")
#Vista que obtiene los detalles de las ventas en las tiendas que registraron los precios mas bajos y mas altos de combustible
#en algun momento del tiempo
sqlContext.sql("""
CREATE VIEW IF NOT EXISTS WeeklySalesByFuelPriceMaxMin AS
WITH MinMaxFuelCTE AS (
    SELECT Min(Fuel_Price) MinFuelPrice, Max(Fuel_Price) MaxFuelPrice
    FROM StoreFeaturesSales
)
SELECT s.Store, s.Date,Fuel_Price,SUM(Weekly_Sales) SumWeeklySales
FROM MinMaxFuelCTE cte
INNER JOIN StoreFeaturesSales s
ON s.Fuel_Price = cte.MinFuelPrice OR s.Fuel_Price = cte.MaxFuelPrice
GROUP BY 1,2,3
ORDER BY 3 DESC,4 DESC
""")
#Vista para obtener las ventas semanales contrastandolas contra la temperatura promedio de la tienda en el mism año
sqlContext.sql("""
CREATE VIEW IF NOT EXISTS WeeklySalesByStoreTemperature AS
SELECT Store, date_format(Date,'yyyy') Year, AVG(Temperature) AvgTemperature, SUM(Weekly_Sales) WeeklySales
FROM StoreFeaturesSales GROUP BY 1,2 ORDER BY 3 DESC
""")
sqlContext.sql("""
SELECT * FROM WeeklySalesByType
""").show()
sqlContext.sql("""
SELECT * FROM WeeklySalesByIsHoliday 
""").show()
sqlContext.sql("""
SELECT * FROM WeeklySalesBySize 
""").show()
sqlContext.sql("""
SELECT * FROM WeeklySalesByFuelPriceMaxMin
""").show()
sqlContext.sql("""
SELECT * FROM WeeklySalesByStoreTemperature
""").show()


+----+--------------+--------------+--------------+--------------+------------------+
|Type|SumWeeklySales|MinWeeklySales|MaxWeeklySales|AvgWeeklySales|    StdWeeklySales|
+----+--------------+--------------+--------------+--------------+------------------+
|   A| 4331014722.75|      -4988.94|     474330.10|  20099.568043| 26423.45722744022|
|   B| 2000700736.82|      -3924.00|     693099.36|  12237.075977| 17203.66898937672|
|   C|  405503527.54|       -379.00|     112152.35|   9519.532538|15985.351612386386|
+----+--------------+--------------+--------------+--------------+------------------+

+---------+--------------+--------------+--------------+
|IsHoliday|MinWeeklySales|MaxWeeklySales|AvgTemperature|
+---------+--------------+--------------+--------------+
|    false|      -4988.94|     406988.63|     60.881518|
|     true|       -798.00|     693099.36|     49.632550|
+---------+--------------+--------------+--------------+

+-----------+--------------+------------------+
|     